In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
import keras
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Dropout
from tensorflow.keras.layers import MaxPooling2D, Add
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, UpSampling2D, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import SGD
from sklearn import model_selection,tree
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [ ]:
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model


#for RESNET


# def model():
# 	base_model = ResNet50(include_top=False, input_shape=(128, 128, 3), pooling='max')
	
# 	for layer in base_model.layers:
# 		layer.trainable = False
# 	model = Sequential()
# 	model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu', input_shape=(32,32,3)))
# 	model.add(BatchNormalization())
# 	model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu'))
# 	model.add(BatchNormalization())
# 	model.add(base_model)
# 	model.add(Flatten())
# 	model.add(Dense(1024, activation='relu'))
# 	model.add(Dense(512, activation='relu'))
# 	model.add(Dense(10, activation='softmax'))
 
#   # opt = SGD(lr=0.001, momentum=0.9)

#   model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	
	# return model

def resnet():
    visible = Input(shape=(32,32,3))
# first feature extractor
    conv1 = Conv2D(64, kernel_size=4, activation='relu',padding='same')(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv1)
    bn1 = BatchNormalization()(pool1)
    print(bn1.shape)
    conv2 = Conv2D(64, kernel_size=4, activation='relu',padding='same')(conv1)
    conv3 = Conv2D(64, kernel_size=4, activation='relu',padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv3)
    bn2 = BatchNormalization()(pool2)
    # print(bn2.shape)
    # conv3 = Conv2D(32, kernel_size=4, activation='relu',padding='same')(bn2)
    # pool3 = MaxPooling2D(pool_size=(2, 2),padding='same')(conv3)
    # bn3 = BatchNormalization()(pool3)
    # print(bn3.shape)



    add1 = Add()([bn1, bn2])

    # conv4 = Conv2D(128, kernel_size=4, activation='relu')(add1)
    # pool4 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv4)
    # bn4 = BatchNormalization()(pool4)

    # conv5 = Conv2D(128, kernel_size=4, activation='relu')(bn4)
    # pool5 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv5)
    # bn5 = BatchNormalization()(pool5)


    flat1 = Flatten()(add1)
    # second feature extractor
    # conv2 = Conv2D(32, kernel_size=8, activation='relu',padding='same')(visible)
    # pool2 = MaxPooling2D(pool_size=(2, 2), ,padding='same')(conv2)
    # flat2 = Flatten()(pool2)
    # merge feature extractors
    # merge = Add()([pool1, pool2])
    # # interpretation layer
    hidden1 = Dense(128, activation='relu')(flat1)
    # prediction output
    output = Dense(10, activation='softmax')(hidden1)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    
    # summarize layers
    # print(model.summary())


In [ ]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def preprocess_image_input(input_images):
	input_images = input_images.astype('float32')
	output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
	return output_ims


    
shadow_models = 3
models = []
hist = []

x_train, y_train, x_test, y_test = load_dataset()
# x_train = preprocess_image_input(x_train)
# x_test = preprocess_image_input(x_test)
x_train, x_test = prep_pixels(x_train, x_test)
train_x = []
train_y = []
for i in range(x_train.shape[0] + x_test.shape[0]):
	
	if i<x_train.shape[0]:
		train_x.append(x_train[i])
		train_y.append(y_train[i])
	else:
		train_x.append(x_test[i-50000])
		train_y.append(y_test[i-50000])

train_x = np.array(train_x)
train_y = np.array(train_y)
print(train_x.shape)
print(train_y.shape)

(60000, 32, 32, 3)
(60000, 10)


In [ ]:
# shadow_models = 2
# models = []
# hist = []

# x_train, y_train, x_test, y_test = load_dataset()
# # x_train = preprocess_image_input(x_train)
# # x_test = preprocess_image_input(x_test)
# x_train, x_test = prep_pixels(x_train, x_test)

attack_x = []




# for i in range(shadow_models):
#     x_data_i = train_x[((i) * 40000) : min(x_train.shape[0], (i+1)*40000)]
#     y_data_i = train_y[((i) * 40000) : min(y_train.shape[0], (i+1)*40000)]
#     # datagen = ImageDataGenerator(
#     # featurewise_center=True,
#     # featurewise_std_normalization=True,
#     # rotation_range=20,
#     # width_shift_range=0.2,
#     # height_shift_range=0.2,
#     # horizontal_flip=True,
#     # validation_split=0.2)
#     x_train_i, x_test_i, y_train_i, y_test_i = model_selection.train_test_split(x_data_i, y_data_i, random_state=17, train_size=0.5)
#     # datagen.fit(x_train_i)

#     model_i = define_model()
#     if i==0:
#         print(model_i.summary())
#     epochs = 10
#     print("Training model: " + str(i+1) + " for data from " + str(i * 40000 ) + " to " + str( min(y_train.shape[0], (i+1)*40000)))
#     # hist_i = model_i.fit(datagen.flow(x_train_i, y_train_i, batch_size=16),epochs=epochs, validation_data=(x_test_i, y_test_i), batch_size=64, verbose=True)
    
#     hist_i = model_i.fit(x_train_i, y_train_i, epochs=200, batch_size=64, verbose=True, validation_data=(x_test_i, y_test_i))
#     models.append(model_i)
#     hist.append(hist_i)
#     preds_test = model_i.predict(x_test_i)
#     preds_train = model_i.predict(x_train_i)

#     for j in range(len(x_train_i)):
#         temp = []
#         for k in range(len(preds_train[j])):
#             temp.append(preds_train[j][k])
        
#         temp.append(np.argmax(y_train_i[j]))
#         temp.append(1)
#         attack_x.append(temp)
        
    
#     for j in range(len(x_test_i)):
#         temp = []
#         for k in range(len(preds_test[j])):
#             temp.append(preds_test[j][k])
        
#         temp.append(np.argmax(y_test_i[j]))
#         temp.append(0)
#         attack_x.append(temp)
    

In [ ]:
for i in range(shadow_models):
  data_init = i * 10000
  data_final = data_init + 40000
  data_x_i = train_x[data_init : data_final]
  data_y_i = train_y[data_init : data_final]
  model_i = define_model()
  if i==0:
      print(model_i.summary())
  epochs = 10
  print("Training model: " + str(i+1) + " for data from " + str(data_init ) + " to " + str( data_final))
  x_train_i, x_test_i, y_train_i, y_test_i = model_selection.train_test_split(data_x_i, data_y_i, random_state=17, train_size=0.5)
  hist_i = model_i.fit(x_train_i,y_train_i, epochs=100, verbose=True, validation_data=(x_test_i, y_test_i))
  models.append(model_i)
  hist.append(hist_i)
  preds_test = model_i.predict(x_test_i)
  preds_train = model_i.predict(x_train_i)

  for j in range(len(x_train_i)):
      temp = []
      for k in range(len(preds_train[j])):
          temp.append(preds_train[j][k])
      
      temp.append(np.argmax(y_train_i[j]))
      temp.append(1)
      attack_x.append(temp)
      

  for j in range(len(x_test_i)):
      temp = []
      for k in range(len(preds_test[j])):
          temp.append(preds_test[j][k])
      
      temp.append(np.argmax(y_test_i[j]))
      temp.append(0)
      attack_x.append(temp)
  


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_19 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_20 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [ ]:
attack_x = np.array(attack_x)


In [ ]:
attack_models_no = 10
attack_models = []
score = []
for i in range(attack_models_no):
    attack_model_i = tree.DecisionTreeClassifier()
    data_i = attack_x[attack_x[:, -2] == i]
    data_x_i = data_i[:, 0:-1]
    data_y_i = data_i[:, -1]
    data_train_x_i, data_test_x_i, data_train_y_i, data_test_y_i = model_selection.train_test_split(data_x_i, data_y_i,train_size=0.3,random_state=17)
    attack_model_i.fit(data_train_x_i, data_train_y_i)
    score.append(attack_model_i.score(data_test_x_i, data_test_y_i))


In [ ]:
np.average(score)


0.568205311447284